# Guardrail Classifier (sürətli və praktik)

Məqsəd:
1) Balanslı training set yaratmaq (`UNSAFE=1`, `SAFE=0`)
2) 1 epoch ilə sürətli training
3) Modelin işlədiyini sanity check ilə göstərmək

Qeyd: Laptop/CPU mühitinə görə parametrləri sürət üçün seçmişəm (yüksək batch, kiçik max_len və az data). Lokala klonlandıqdan sonra laptop göstəricilərindən asılı olaraq parametrlərdə dəyişiklik edə bilərsiniz.

In [1]:
from pathlib import Path
import os, sys, subprocess

ROOT = Path.cwd()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

os.chdir(ROOT)
print("Repo root:", Path.cwd())

PYTHONPATH = str(Path.cwd() / "src")
ENV = os.environ.copy()
ENV["PYTHONPATH"] = PYTHONPATH + (os.pathsep + ENV["PYTHONPATH"] if ENV.get("PYTHONPATH") else "")

def run(cmd):
    """Always run with the same python as this notebook kernel + correct PYTHONPATH."""
    if isinstance(cmd, str):
        cmd = cmd.split()
    print("RUN:", " ".join(cmd))
    return subprocess.run(cmd, env=ENV, check=True)


Repo root: d:\github_repos\kontakt_home_task\task3


## Bu bootstrap nəyə lazımdır?

Notebook-lar `task3/notebooks/` içində olduğu üçün, birbaşa işlətsək `scripts/...` və `src/...` yolları tapılmaya bilər.

Bu blok nə edir?

- **Repo root-a keçir (`task3/`)** — yollar düz işləsin deyə  
- **`PYTHONPATH=src` verir** — `import pii_guard...` xətası çıxmasın deyə  
- **`sys.executable` istifadə edir** — notebook-un istifadə etdiyi Python ilə eyni interpreter işləsin deyə  

Yəni “məndə işləyir, səndə işləmir” tipli problemləri azaldır :D


## Dataset building (8k unsafe + 8k safe)

In [2]:
run([sys.executable, "scripts/build_train_classifier_json.py", "--filter_safe_pii", "--max_unsafe", "4000"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe scripts/build_train_classifier_json.py --filter_safe_pii --max_unsafe 4000


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', 'scripts/build_train_classifier_json.py', '--filter_safe_pii', '--max_unsafe', '4000'], returncode=0)

Script-in nəticəsi: [data/processed/train_classifier.json](../data/processed/train_classifier.json)

Bu faylda hər sətir belədir:
- `text`  → cümlə
- `label` → 0 (SAFE) / 1 (UNSAFE)
- `split` → train / validation

Yəni növbəti mərhələdə modelə “hazır dataset” veririk.

## Train classifier (1 epoch, batch 256, max_len)
256 seçmə səbəbimiz modelin işləkliyini vaxt itirmədən icmali şəkildə göstərməkdir.

In [4]:
run([sys.executable, "-m", "pii_guard.training.train_classifier",
     "--epochs", "1", "--batch", "256", "--max_len", "24"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe -m pii_guard.training.train_classifier --epochs 1 --batch 256 --max_len 24


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', '-m', 'pii_guard.training.train_classifier', '--epochs', '1', '--batch', '256', '--max_len', '24'], returncode=0)

## Quick sanity inference (PyTorch)

In [5]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
model_dir = "models/classifier/pytorch"
tok = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
model.eval()

The tokenizer you are loading from 'models/classifier/pytorch' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [7]:
samples = [
    "Salam, sabah görüşərik.",
    "Mənim telefon nömrəm +994 50 123 45 67-dir.",
    "Kartım 4169 1234 5678 9012, adım Aysel Mammadova.",
]

In [8]:
with torch.no_grad():
    for s in samples:
        enc = tok(s, return_tensors="pt", truncation=True, max_length=96)
        logits = model(**enc).logits[0].cpu().numpy()
        exps = np.exp(logits - logits.max())
        probs = exps / exps.sum()
        print(f"UNSAFE prob={float(probs[1]):.3f} | {s}")

UNSAFE prob=0.661 | Salam, sabah görüşərik.
UNSAFE prob=0.895 | Mənim telefon nömrəm +994 50 123 45 67-dir.
UNSAFE prob=0.876 | Kartım 4169 1234 5678 9012, adım Aysel Mammadova.
